##### Information-theretic analysis of language models (Fall 2022/3)


# Home Assignment 2

#### Topics:
- Lossless compression

#### Due: 14/12/2022 before the class

#### Instructions:
- Write your name, Student ID, and date in the cell below. 
- Submit a copy of this notebook with code filled in the relevant places as the solution of coding excercises.
- For theoretic excercises, you can either write your solution in the notebook using $\LaTeX$ or submit additional notes.

<hr>
<hr>


**Name**: 

**Student ID**:

**Date**:

$
\newcommand{\Id}{{\mathbf{I}}}  
\newcommand{\SSE}{\mathsf{SSE}}
\newcommand{\SSR}{\mathsf{SSR}}
\newcommand{\MSE}{\mathsf{MSE}}
\newcommand{\simiid}{\overset{iid}{\sim}}
\newcommand{\ex}{\mathbb E}
\newcommand{\var}{\mathrm{Var}}
\newcommand{\Cov}[2]{{\mathrm{Cov}  \left(#1, #2 \right)}}
\newcommand{\one}[1]{\mathbf 1 {\left\{#1\right\}}}
\newcommand{\SE}[1]{\mathrm{SE} \left[#1\right]}
\newcommand{\reals}{\mathbb R}
\newcommand{\Ncal}{\mathcal N}
\newcommand{\abs}[1]{\ensuremath{\left\vert#1\right\vert}}
\newcommand{\rank}{\operatorname{rank}}
\newcommand{\tr}{\operatorname{Tr}}
\newcommand{\diag}{\operatorname{diag}}
\newcommand{\sign}{\operatorname{sign}}
\newcommand{\Ycal}{\mathcal Y}
\newcommand{\Xcal}{\mathcal X}
\newcommand{\Zcal}{\mathcal Z}
\newcommand{\Wcal}{\mathcal W}
$


## 1. Compressing a Markov Source
In this question you will sample a sequence from a two-states Markov source and compress this sequence in a losslessly manner using several methods. The function ``sample_Markov_path`` below samples such a sequence. 

Use the transition matrix 
$$
Q = \begin{bmatrix} 1-\alpha & \alpha \\
\beta & 1- \beta
\end{bmatrix}
$$
and vector of initial probabilities $\begin{bmatrix} 1, 0 \end{bmatrix}$ (namely, you begin at state $0$). 


In [88]:
YOUR_ID_HERE = 1

In [252]:
import numpy as np
from scipy.stats import multinomial
from matplotlib import pyplot as plt

SEED = YOUR_ID_HERE

def sample_Markov_path(Q: np.ndarray, initial_probs: np.ndarray, n: int)->np.ndarray:
    """
    Sample from a path from a Markov chain
    
    Args:
        :Q:  transition probability matrix
        :initial_probs:  vector of probabilities of the initial state
        :n:  length of path
    
    Return:
        :xx:  sample from the Markov chain of length n
        
    """

    M = Q.shape[0]
    xx = np.zeros((n,M))

    prob_vec = initial_probs

    for i in range(n):
        xx[i] = multinomial.rvs(p=prob_vec, n=1, random_state=SEED+i)
        prob_vec = xx[i] @ Q

    return np.argmax(xx, 1)



A short sample from the Markov chain (set $n = 2^{14}$ when solving the assignment):

In [253]:
alpha = 0.1
beta = 0.05

Q = np.array([
    [1-alpha, alpha],
    [beta, 1-beta] 
])

initial_probs = [1, 0]  # start at state 0
X = sample_Markov_path(Q, initial_probs, n = 100)

print(X)

[0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


(1) What is the entropy rate of this source? is it smaller or larger than the entropy of the stationary distribution?

(2) Set the ``SEED`` as your id number. With $\alpha=.1$ and $\beta=.05$, generate a binary string of length $n=2^{14}$ from this soruce (using the function ``sample_Markov_path``). What is the fraction of times you spend at each state? verify that this fraction matched more or less the stationary distribution. 

(3) Compress the binary string using a Huffman code for tuples of 8 symbols (one byte), i.e., consider the tensorized source with $K=8$. Estimate tuple frequenceies either from the data or directly from the model. Plot the frequencies of the $2^K$ tuples. Can you anticipate the compression rate (``bits_compressed`` / ``bits_original``) without actually do the encoding?  

(4) Compress the binary string using Run Length Encoding (RLE) with a maximal stretch of $2^k$. Namely, for $k=3$, the string 000001100011111111.... is encoded as (0,4), (1,1), (0,2), (1,7)..., which is then encoded as (0,100), (1,001), (0,010), (1,111), which is then encoded as 0100 1001 0010 1111 (each stretch of "1"s or "0"s is encoded using $1+k$ bits. We subtract one from the length of the stretch because there are no stretches of length 0). Stretches longer than $2^k$ are seperated into a stretch of $2^k$ and the remainder. Experiment with values of $k$ between 2-8 and report the compression rate for each $k$. Which $k$ attains the best compression rate?

(5) Bonus: Can you think about a way to improve the proposed RLE?

## 2. Guessing game and compression
In this question you should work with a simplified version of the text of pride and prejudice: lower case, no punctuation but with sentence limits. You can use the function ``simplify_text`` available below.

(1) Build a word-bigram model based on the data; use this model to transform the text into a sequence of guess numbers. Plot the histogram of the sequence of guess numbers and report the entropy of the empirical distribution this histogram represents. 

(2) Build a decoder that recovers the original text from the sequence of guesses; conclude that guessing + model is an invertible transformation. 

(3) Compress the sequence of guesses using a Huffman code. What is the average number of bits per **letter**?

In [256]:
import numpy as np
import re

TOKEN_PATTERN = r"(?u)[a-zA-Z]+|\</?s\>"
SENT_START_TOKEN = '<s>'
SENT_END_TOKEN = '</s>'


def to_tokens(text: str) -> list:
    return re.findall(TOKEN_PATTERN, text.lower())


def normalize_text(text: str) -> str:
    """
    Remove/add dots to indicate sentence limits
    """
    
    text = re.sub("(Mrs?)\.", "\\1", text) # Mr/s. -> Mr/s
    text = re.sub("[!?]", ".", text) # !? -> .
    text = re.sub("(I+)\.", "\\1", text) # II. -> II
    text = re.sub("([a-zA-Z])\.([a-zA-Z])\.", "\\1\\2", text) #i.e.->ie e.g. -> eg
    return text

def add_sentence_limits(text: str, sep=r'\.') -> str:
    """
    Add SENT_START_TOKEN and SENT_END_TOKEN at the beginning and
    ending of every sentence. 
    
    Args:
        :text: is a text input
        :sep: explains how to identify sentnce ending (regular expression)
    """
    sentences = re.split(sep, normalize_text(text))
    sent_break = f' {SENT_END_TOKEN} {SENT_START_TOKEN} '
    return SENT_START_TOKEN + ' ' + sent_break.join(sentences) + ' ' + SENT_END_TOKEN


def simplify_text(text: str) -> str:
    """
    Returns a simplified version of the text:
     - lower case 
     - sentence limits marking
     - no punctuation or new lines
    """
    return " ".join(to_tokens(add_sentence_limits(text)))